In [1]:
import pandas as pd
import numpy as np

In [2]:
def resumen(datos):
    return print('Forma:\n', datos.shape,'\n\n',
                 'Columnas:\n', datos.columns,'\n\n',
                 'Tipos:\n', datos.dtypes,'\n\n',
                 'Datos nulos:\n', datos.isnull().sum(),'\n\n',
                 'Cabecera:\n', datos.head(10),'\n\n',
                 'Últimos datos:\n', datos.tail(10),'\n\n',
                 'Estadísticos:\n', datos.describe([0.05,0.25,0.5,0.75,0.95])
                )
def truncar(numero, decimales = 0):
    return np.trunc(numero*10**decimales)/(10**decimales)

In [3]:
def presion_saturacion(t):
    # Poner t en °C y devuelve e en HPa
    e = 6.11*np.exp(6808*((1/273.15)-(1/(t+273.15)))-5.09*np.log((t+273.15)/273.15))
    return e
def densidad_aire(t, p):
    # Poner t en °C y p en HPa
    ro = (p*100) / (287.058*(t+273.3))
    return ro
def humedad_especifica(a, ro):
    q = a/ro
    return q
def presion_vapor(q, p):
    # Poner q en gr/kg
    e = p*q / (622 + q)
    return e

In [4]:
datos_juntos = pd.read_csv(
    'C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\Datos_juntos.csv',
    delimiter = '\t',
    decimal = '.',
    na_values = -9999,
    encoding = 'ascii'
    )

In [5]:
resumen(datos_juntos)

Forma:
 (29914, 124) 

 Columnas:
 Index(['Momento', 'TSS1', 'TSS2', 'TS1', 'TS2', 'TS3', 'TS4', 'TS_QG', 'U',
       'DIR',
       ...
       ' r_err_ustar[%]', '   r_err_HTs[%]', '   r_err_LvE[%]',
       '   r_err_co2[%]', ' noise_ustar[%]', '   noise_HTs[%]',
       '   noise_LvE[%]', '   noise_co2[%]', 'Unnamed: 61', 'anio'],
      dtype='object', length=124) 

 Tipos:
 Momento            float64
TSS1               float64
TSS2               float64
TS1                float64
TS2                float64
TS3                float64
TS4                float64
TS_QG              float64
U                  float64
DIR                float64
PP                 float64
TA1                float64
HR1                float64
TA2                float64
HR2                float64
TA3                float64
HR3                float64
TA4                float64
HR4                float64
QG1                float64
QG2                float64
WET                float64
RG                 float64
P

In [6]:
datos_juntos.loc[-1] = [np.nan] * 124
datos_juntos.index = datos_juntos.index + 1
datos_juntos.sort_index(inplace = True)
col = [1, 2, 3, 4]
total = list(range(len(datos_juntos['TSS1'])))
for i in col:
    num = str(col[i-1])
    datos_juntos['e_s ' + num] = total[:]
    for j in total:
        datos_juntos['e_s ' + num][j] = presion_saturacion(datos_juntos['TA' + num][j])
    datos_juntos['VPD ' + num] = round(
        datos_juntos['e_s ' + num] * (1 - datos_juntos['HR' + num]/100),
        2
        )
datos_juntos['e_s Ts'] = total[:]
for k in total:
    datos_juntos['e_s Ts'][k] = presion_saturacion(datos_juntos['Ts[degC]       '][k])
rho = densidad_aire(datos_juntos['Ts[degC]       '],
                    datos_juntos['p_ref[hPa]     ']
                   )
q = humedad_especifica(datos_juntos['a[g/m3]        '],
                      rho
                      )
e = presion_vapor(q, datos_juntos['p_ref[hPa]     '])
datos_juntos['VPD Ts'] = round(
    datos_juntos['e_s Ts'] - e,
    2
    )

C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\nahue\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [7]:
datos_juntos.head(20)

,Momento,TSS1,TSS2,TS1,TS2,TS3,TS4,TS_QG,U,DIR,...,e_s 1,VPD 1,e_s 2,VPD 2,e_s 3,VPD 3,e_s 4,VPD 4,e_s Ts,VPD Ts
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,54.36458,18.15,17.97,17.89,19.35,19.21,18.66,18.47,4.701,165.8,...,21.945891,8.68,22.865282,9.09,22.429336,8.68,22.138164,8.74,21.582729,3.48
2,54.37500,18.16,18.12,18.10,19.35,19.21,18.67,18.46,5.088,166.4,...,22.193369,8.89,23.351919,9.55,22.836934,9.06,22.387536,8.96,21.796771,3.75
3,54.38542,18.18,18.26,18.41,19.32,19.20,18.67,18.46,5.180,166.0,...,22.499154,9.30,23.715494,9.99,23.236609,9.55,22.681572,9.35,22.208724,4.12
4,54.39583,18.25,18.35,18.80,19.30,19.19,18.67,18.46,5.217,170.4,...,22.597219,9.69,24.039523,10.58,23.453226,10.05,22.808618,9.76,22.320836,4.54
5,54.40625,18.35,18.53,19.07,19.26,19.16,18.65,18.47,5.059,156.8,...,22.723852,9.58,24.322473,10.63,23.730140,10.11,22.964741,9.67,22.314838,4.20
6,54.41667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.481259,4.50
7,54.42708,19.00,19.02,19.98,19.26,19.16,18.64,18.63,4.685,154.7,...,22.836934,9.58,24.897116,11.08,23.936006,10.25,23.050292,9.68,22.665825,4.54
8,54.43750,19.26,19.16,20.67,19.28,19.17,18.65,18.72,4.681,157.2,...,22.964741,9.99,24.866578,11.37,24.217892,10.73,23.265390,10.14,22.706276,4.69
9,54.44792,19.44,19.36,21.23,19.28,19.17,18.63,18.78,5.068,165.6,...,22.836934,9.63,24.668882,10.98,24.024711,10.36,23.107481,9.78,22.539700,4.59


In [8]:
col = datos_juntos.columns[-10:]
datos_juntos[col].sample(20)

,e_s 1,VPD 1,e_s 2,VPD 2,e_s 3,VPD 3,e_s 4,VPD 4,e_s Ts,VPD Ts
18576,14.643373,3.25,13.920694,14.25,14.170448,3.26,14.739604,3.37,14.629559,11.86
29066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19070,12.147298,1.31,12.033793,12.32,12.090428,1.60,12.269956,1.51,12.194014,NaN
3857,29.438707,12.82,31.440714,13.92,30.846433,12.94,29.580463,12.50,30.128072,7.19
19225,18.512893,5.80,19.675311,20.14,19.354345,6.87,18.773825,5.92,18.604482,-0.18
7487,10.081993,0.88,10.109659,1.01,10.137392,4.90,10.277065,1.09,NaN,NaN
18581,14.235838,2.41,13.893184,14.22,14.040459,2.61,14.367415,2.57,13.710901,NaN
17310,15.159761,6.73,15.590402,15.96,15.539782,7.26,15.368751,6.85,14.841906,1.75
26555,20.640409,3.96,18.301773,18.73,18.821612,3.22,20.178212,4.19,19.073842,-2.04
22150,12.082323,3.59,11.881244,12.16,11.921222,3.74,12.090428,3.71,11.348685,-1.88


In [10]:
datos_juntos.to_csv('C:\\Users\\nahue\\Desktop\\Tesis_2\\Datos\\Datos_juntos.csv',
                       sep = '\t',
                       na_rep = -9999,
                       index = False,
                       encoding = 'ascii'
                       )